<a href="https://colab.research.google.com/github/ysys143/ml2024/blob/main/finding_news_v0_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [342]:
from google.colab import drive
drive.mount('/content/drive')

news_db = pd.read_csv('/content/drive/MyDrive/2024-NIPA-google-trendpop/news_bot/news_db.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [343]:
!pip install -q langchain #언어모델을 활용한 어플리케이션 제작에 도움을 주는 프레임워크
!pip install -q -U langchain-community #랭체인에서 제공하는 추가 기능

!pip install -q langchain_openai #gemini 언어모델
!pip install -q sentence-transformers #문장을 벡터로 변환하는데 사용하는 라이브러리
!pip install -q unstructured #비정형 데이터를 처리하고 구조화된 형태로 변환하는 데 사용됨

!pip install -q tiktoken # 토큰화
!pip install -q faiss-cpu # 벡터 검색, 유사도 검색

In [258]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

In [274]:
import os
import pandas as pd
import openai
import json
import requests
import faiss
from datetime import datetime, timedelta

from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredURLLoader
from langchain.docstore.document import Document

from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

from langchain.utilities import GoogleSerperAPIWrapper

In [260]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('openai_key')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
embeddings = OpenAIEmbeddings()

In [344]:
!pip install --upgrade nltk

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 유사성 검색

In [345]:
# 함수 정의: query, news_db(dataframe), k을 받아 유사한 k개 타이틀 반환
from sklearn.metrics.pairwise import cosine_similarity
import numpy

embeddings = OpenAIEmbeddings()

def similarity_search(news_db, query, k=3):

    # 데이터셋 크기보다 k가 클 경우, k를 데이터셋 크기로 설정
    if len(news_db) < k:
        k = len(news_db)

    # query를 임베딩
    #query_embedding = model.encode([query])
    query_embedding = np.array(embeddings.embed_query(query)).reshape(1, -1)  # 2D로 변환

    # news_db의 title을 임베딩
    #title_embeddings = model.encode(news_db['title'].tolist())
    title_embeddings = np.array(embeddings.embed_documents(news_db['title'].tolist()))

    # 코사인 유사도를 계산
    similarities = cosine_similarity(query_embedding, title_embeddings).flatten()

    # 유사도가 높은 k개의 인덱스를 선택
    top_k_indices = similarities.argsort()[-k:][::-1]

    # 해당 인덱스에 해당하는 행을 반환
    return news_db.reset_index().iloc[top_k_indices]

In [346]:
# OpenAI 임베딩 모델 정의
embeddings = OpenAIEmbeddings()

def similarity_search2(news_db, query, k=3):
    # news_db의 타이틀을 문서 리스트로 변환 (news_db 인덱스를 metadata에 저장)
    docs = []
    for idx, row in news_db.iterrows():
        doc = Document(page_content=row['title'], metadata={"index": idx})
        docs.append(doc)

    # FAISS 인덱스 생성
    db = FAISS.from_documents(docs, embeddings)

    # query와 유사한 문서 k개 검색
    results = db.similarity_search(query, k=k)

    # FAISS 결과에서 metadata의 index 값을 사용해 news_db에서 데이터를 가져옴
    result_indices = [int(result.metadata['index']) for result in results]
    similar_articles = news_db.iloc[result_indices]

    return similar_articles

In [347]:
# Sentence-Transformers 임베딩 모델 정의
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def similarity_search3(news_db, query, k=3):
    # 타이틀에 대한 임베딩 생성
    title_embeddings = model.encode(news_db['title'].tolist())

    # FAISS 인덱스 생성 (코사인 유사도를 위한 L2 노말라이제이션 사용)
    d = title_embeddings.shape[1]  # 벡터 차원 수
    index = faiss.IndexFlatL2(d)   # L2 거리 기반 인덱스 생성
    faiss.normalize_L2(title_embeddings)  # 코사인 유사도를 위한 벡터 정규화

    # 인덱스에 임베딩 추가
    index.add(np.array(title_embeddings, dtype=np.float32))

    # 쿼리 임베딩 생성 및 정규화
    query_embedding = model.encode([query])
    faiss.normalize_L2(query_embedding)

    # FAISS 인덱스를 사용하여 상위 k개의 유사한 인덱스 검색
    distances, indices = index.search(np.array(query_embedding, dtype=np.float32), k)

    # 검색된 인덱스를 사용하여 해당 타이틀에 해당하는 행을 반환
    similar_articles = news_db.iloc[indices.flatten()]

    return similar_articles

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [348]:
# 예시 query 사용
query = "뉴진스 논란"
k = 3
result_df = similarity_search(news_db, query, k)
result_df

,index,date,group,title,link
1109,1109,2023.4.1,NewJeans,뉴진스 '홈마'의 활동중단 입장문 논란,http://www.mediaus.co.kr/news/articleView.html...
9,9,2024.10.15,NewJeans,"뉴진스 하니, 국회 국감장에...'따돌림 논란' 증언",https://www.ytn.co.kr/_ln/0106_202410151415560865
1681,1681,2024.1.17,NewJeans,"뉴진스 민지 사과하게 한, '칼국수 논란'의 전말",http://news.maxmovie.com/436209


In [349]:
# 특정 그룹과 날짜 전후 15일 이내의 데이터프레임을 반환하는 함수
def filter_by_group_and_date(news_db, group, target_date):

    # 날짜 형식을 datetime으로 변환
    news_db['date_datetime'] = pd.to_datetime(news_db['date'], format="%Y.%m.%d")

    # 입력 받은 날짜를 datetime으로 변환
    try:
        # "Y.m.d" 형식 먼저 시도
        target_date_obj = datetime.strptime(target_date, "%Y.%m.%d")
    except ValueError:
        try:
            # "Y-m-d" 형식 시도
            target_date_obj = datetime.strptime(target_date, "%Y-%m-%d")
        except ValueError:
            # 두 형식 모두 실패할 경우 오류 발생
            raise ValueError("Invalid date format. Please use 'YYYY.MM.DD' or 'YYYY-MM-DD'.")

    # 날짜 범위 계산 (target_date_obj 기준 ±15일)
    start_date = target_date_obj - timedelta(days=15)
    end_date = target_date_obj + timedelta(days=15)

    # 조건에 맞는 데이터 필터링
    filtered_df = news_db[
        (news_db['group'] == group) &
        (news_db['date_datetime'] >= start_date) &
        (news_db['date_datetime'] <= end_date)
    ]

    return filtered_df[['date', 'group', 'title', 'link']]


In [351]:
filter_by_group_and_date(news_db, 'NewJeans', '2023.07.13')

,date,group,title,link
1254,2023.6.28,NewJeans,뉴진스 신곡 뮤비 티저 공개되자마자 뜻밖의 표절 논란(?) 터졌다,https://www.insight.co.kr/news/442756
1257,2023.6.30,NewJeans,챗GPT'발 AI 저작권 논란…데이터 임의수집·사용 이대로 좋은가,http://www.goodkyung.com/news/articleView.html...
1262,2023.7.2,NewJeans,"“완전 똑같은데…” '뉴진스' 신곡, 이번에도 예측했다는 '무한도전'",https://www.wikitree.co.kr/articles/866341
1264,2023.7.6,NewJeans,“대형 기획사보다 좋은데…” '뉴진스'보다 좋은 숙소 썼다는 '피프티...,https://www.wikitree.co.kr/articles/867630
1266,2023.7.10,NewJeans,"[라이브] 하이브, 방탄소년단 '군백기'에도 뉴진스 컴백으로 주가 '훨훨...",https://news.sbs.co.kr/news/endPage.do?news_id...
1267,2023.7.10,NewJeans,"하이브, 아이돌 팬사인회서 속옷검사…팬들 “인권 바닥”",https://www.hani.co.kr/arti/society/society_ge...
1272,2023.7.12,NewJeans,"엑소 수호 ""조금 실수해도 조롱""…뮤지컬 '실력 논란' 심경 토로｜상클...",https://news.jtbc.co.kr/article/article.aspx?n...
1273,2023.7.15,NewJeans,[초점] '중소돌' 피프티 피프티 논란에 삼성 '갑툭튀'…'뉴진스' 막을 무...,http://www.inews24.com/view/1612872
1274,2023.7.16,NewJeans,”부끄러움은 3초” 사직 아이돌이 ‘사직 제니’로 변신하기까지…그...,http://www.mydaily.co.kr/new_yk/html/read.php?...
1293,2023.7.21,NewJeans,[뮤직IS] 뉴진스는 항상 특별했다…‘겟 업’으로 물들일 여름 ①,https://isplus.com/article/view/isp202307200010


In [352]:
def create_query(group, date):

    # 문자열을 datetime 객체로 변환
    date_obj = datetime.strptime(date, '%Y-%m-%d')

    # 연도와 월 추출
    year = date_obj.year
    month = date_obj.month

    return f'{year}년 {month}월 {group} 관련 사건사고'

# 날짜 차이를 계산하는 함수
def calculate_date_diff(row, query_date):
    query_date = datetime.strptime(query_date, "%Y-%m-%d")
    event_date = datetime.strptime(row['date'], "%Y.%m.%d")
    return abs((query_date - event_date).days)

# 가장 가까운 사건 k개를 리턴하는 함수
def find_closest_events(news_db, group, date, k):
    # 입력된 날짜 변환
    query_date = datetime.strptime(date, "%Y-%m-%d")

    # 특정 그룹의 사건에 대해 날짜 차이를 계산하여 새로운 열에 추가
    df_filtered = news_db[news_db['group'] == group].copy()
    df_filtered['date_diff'] = df_filtered.apply(lambda row: calculate_date_diff(row, date), axis=1)

    # 날짜 차이가 적은 순으로 정렬 후 상위 k개 선택
    df_sorted = df_filtered.sort_values('date_diff').head(k)

    # 필요한 정보만 반환
    return df_sorted[['date', 'title', 'link']]

In [353]:
def cleaning_article(page_content):
    """
    LLM을 사용하여 불필요한 텍스트를 제거하고 기사 본문만 추출하는 함수
    """
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    # 기사 내용 필터링을 위한 프롬프트
    prompt = f"""
    다음 텍스트는 뉴스 기사 전체입니다. 이 중에서 기사 본문에 해당하지 않는 부분 (예: 날씨, 저작권 정보, 사이트 메뉴 등)을 제거하고, 기사 본문만 남겨줘.

    텍스트:
    {page_content}

    결과:
    """

    response = llm.invoke(prompt)

    # 결과를 직접 content에서 추출
    return response.content.strip()

In [354]:
def summarizer(query, articles):

    # LLM 준비
    llm = ChatOpenAI(model="gpt-4o", temperature=0)

    template = """
    {docs}

    너는 음악산업에 오래 종사해온 프로듀서야.
    KPOP 그룹에 대한 긍부정 감정에 영향을 미치는 여론 동향을 분석하기 위한 흥미롭고, 유용한 아티클을 찾고 있어.
    첨부한 기사를 바탕으로 보고서에 들어갈 요약문을 작성해줘.

    요약문은 아래의 가이드라인에 맞춰서 적어줘.

    1. 콘텐츠로부터 정보를 얻을 수 있고 유용해야 해.
    2. 콘텐츠는 읽기 쉽게 쓰여야 하고, 간결해야 해.
    3. 콘텐츠가 너무 짧거나 길지 않고 144자 정도를 유지해야 해.
    4. "{query}"에 관련된 내용을 잘 나타내고 있는 내용이어야 해.
    5. 긍정적이거나 부정적인 영향을 미칠 수 있을만한 내용이어야 해.

    읽기 편하게 '입니다'체의 한국어로 작성해 줘. 답변에 가이드라인의 내용이 포함되지 않도록 할 것.

    SUMMARY :
    """

    prompt_template = PromptTemplate(input_variables=["docs", "query"], template=template)

    # 각 기사별로 따로 요약 생성
    summaries = []

    for idx, row in articles.iterrows():
        title = row['title']
        date = row['date']
        url = row['link']
        loader = UnstructuredURLLoader([url])
        data = loader.load()
        page_content = data[0].page_content
        #cleaned_content = cleaning_article(page_content) #사용하지 않아도 출력이 잘 나옴

        summarize_chain = prompt_template | llm
        response = summarize_chain.invoke({"docs": page_content, "query": query})
        summary = response.content.strip()

        # 불필요한 "SUMMARY :" 부분 제거하는 후처리 로직 추가
        if "SUMMARY :" in summary:
            summary = summary.replace("SUMMARY :", "").strip()

        formatted_output = f"{date}\n[{title}]\n{url}\n{summary}"
        summaries.append(formatted_output)

    # 각 요약을 합쳐 최종 결과로 반환
    return "\n\n\n".join(summaries)

## test

In [356]:
# 예시 질문
group = "NewJeans"
question_date = "2023-07-05"
k = 3

# 질문 생성
query = create_query(group, question_date)

# group, date로 검색대상범위 지정
filtered_df = filter_by_group_and_date(news_db, group, question_date)

# 가장 가까운 사건과 기사 찾기
#articles = find_closest_events(filtered_df, group, question_date)
articles = similarity_search(filtered_df, query, k=3)

# 요약답변 생성
summarizer(query, articles)

'2023.6.22\n[불붙는 7월 아이돌 컴백대전...틴탑·엑소부터 뉴진스·있지까지 ①]\nhttps://isplus.com/article/view/isp202306210134\n2023년 7월, NewJeans는 독창적인 음악 스타일과 무대 퍼포먼스로 주목받았습니다. 그러나 일부 멤버의 발언이 논란을 일으켜 팬들 사이에서 긍정과 부정의 여론이 형성되었습니다. 이러한 사건은 그룹의 이미지에 다양한 영향을 미칠 수 있습니다.\n\n\n2023.6.28\n[뉴진스 신곡 뮤비 티저 공개되자마자 뜻밖의 표절 논란(?) 터졌다]\nhttps://www.insight.co.kr/news/442756\n2023년 7월, 뉴진스의 신곡 \'ASAP\' 뮤비 티저가 공개되자 \'무한도전\'과의 유사성으로 표절 논란이 제기되었습니다. 이 논란은 팬들 사이에서 웃음을 자아내며 긍정적인 반응을 얻고 있습니다. 티저 영상은 하루 만에 347만 뷰를 기록하며 큰 인기를 끌고 있습니다.\n\n\n2023.7.15\n[[초점] \'중소돌\' 피프티 피프티 논란에 삼성 \'갑툭튀\'…\'뉴진스\' 막을 무...]\nhttp://www.inews24.com/view/1612872\n죄송하지만, 첨부된 기사를 확인할 수 없습니다. 대신, "2023년 7월 NewJeans 관련 사건사고"에 대한 정보를 제공해 주시면, 그에 맞춰 요약문을 작성해 드리겠습니다.'